In [1]:
import os
import pathlib
if "models" in pathlib.Path.cwd().parts:
  while "models" in pathlib.Path.cwd().parts:
    os.chdir('..')
elif not pathlib.Path('models').exists():
  !git clone --depth 1 https://github.com/tensorflow/models

In [ ]:
# instal tensorflow models from here 
# https://github.com/tensorflow/models.git

In [3]:
!pip install tensorflow

  Using cached tensorflow-2.8.0-cp39-cp39-win_amd64.whl (438.0 MB)
  Using cached flatbuffers-2.0-py2.py3-none-any.whl (26 kB)
  Using cached gast-0.5.3-py3-none-any.whl (19 kB)
  Using cached opt_einsum-3.3.0-py3-none-any.whl (65 kB)
  Using cached termcolor-1.1.0-py3-none-any.whl
  Using cached absl_py-1.0.0-py3-none-any.whl (126 kB)
  Using cached tensorflow_io_gcs_filesystem-0.24.0-cp39-cp39-win_amd64.whl (1.5 MB)
  Using cached grpcio-1.44.0-cp39-cp39-win_amd64.whl (3.4 MB)
  Using cached libclang-13.0.0-py2.py3-none-win_amd64.whl (13.9 MB)
  Using cached Keras_Preprocessing-1.1.2-py2.py3-none-any.whl (42 kB)
  Using cached keras-2.8.0-py2.py3-none-any.whl (1.4 MB)
  Using cached tensorboard-2.8.0-py3-none-any.whl (5.8 MB)
  Using cached google_pasta-0.2.0-py3-none-any.whl (57 kB)
  Using cached tf_estimator_nightly-2.8.0.dev2021122109-py2.py3-none-any.whl (462 kB)
  Using cached astunparse-1.6.3-py2.py3-none-any.whl (12 kB)
  Using cached tensorboard_plugin_wit-1.8.1-py3-none-any

In [4]:
import numpy as np
import os
import six.moves.urllib as urllib
import sys
import tarfile
import tensorflow as tf
import zipfile
import pathlib
from collections import defaultdict
from io import StringIO
from matplotlib import pyplot as plt
from PIL import Image
from IPython.display import display

In [6]:
!pip install tensorflow-object-detection-api

  Using cached tensorflow_object_detection_api-0.1.1-py3-none-any.whl
  Using cached rfc3986-2.0.0-py2.py3-none-any.whl (31 kB)
  Using cached pkginfo-1.8.2-py2.py3-none-any.whl (26 kB)
  Using cached requests_toolbelt-0.9.1-py2.py3-none-any.whl (54 kB)
  Using cached readme_renderer-34.0-py3-none-any.whl (16 kB)
  Attempting uninstall: pkginfo
    Found existing installation: pkginfo 1.7.1


    Uninstalling pkginfo-1.7.1:
      Successfully uninstalled pkginfo-1.7.1


In [7]:
from object_detection.utils import ops as utils_ops
from object_detection.utils import label_map_util
from object_detection.utils import visualization_utils as vis_util

In [8]:
 # patch tf1 into `utils.ops`
utils_ops.tf = tf.compat.v1

# Patch the location of gfile
tf.gfile = tf.io.gfile

In [1]:
def load_model(model_name):
    base_url = 'http://download.tensorflow.org/models/object_detection/'
    model_file = model_name + '.tar.gz'
    model_dir = tf.keras.utils.get_file(
    fname=model_name, 
    origin=base_url + model_file,
    untar=True)
    model_dir = pathlib.Path(model_dir)/"saved_model"
    model = tf.saved_model.load(str(model_dir))
    model = model.signatures['serving_default']
    return model

In [2]:
PATH_TO_LABELS = 'models/research/object_detection/data/mscoco_label_map.pbtxt'
category_index = label_map_util.create_category_index_from_labelmap(PATH_TO_LABELS, use_display_name=True)

NameError: name 'label_map_util' is not defined

In [11]:
PATH_TO_TEST_IMAGES_DIR = pathlib.Path('models/research/object_detection/test_images')
TEST_IMAGE_PATHS = sorted(list(PATH_TO_TEST_IMAGES_DIR.glob("*.jpg")))
TEST_IMAGE_PATHS

[WindowsPath('models/research/object_detection/test_images/12.jpg'),
 WindowsPath('models/research/object_detection/test_images/13.jpg'),
 WindowsPath('models/research/object_detection/test_images/14.jpg'),
 WindowsPath('models/research/object_detection/test_images/15.jpg'),
 WindowsPath('models/research/object_detection/test_images/16.jpg'),
 WindowsPath('models/research/object_detection/test_images/17.jpg'),
 WindowsPath('models/research/object_detection/test_images/18.jpg'),
 WindowsPath('models/research/object_detection/test_images/9.jpg'),
 WindowsPath('models/research/object_detection/test_images/image1.jpg'),
 WindowsPath('models/research/object_detection/test_images/image2.jpg'),
 WindowsPath('models/research/object_detection/test_images/image3.jpg')]

In [12]:
model_name = 'ssd_mobilenet_v1_coco_2017_11_17'
detection_model = load_model(model_name)

INFO:tensorflow:Saver not created because there are no variables in the graph to restore


In [13]:
print(detection_model.inputs)

[<tf.Tensor 'image_tensor:0' shape=(None, None, None, 3) dtype=uint8>]


In [14]:
detection_model.output_dtypes

{'detection_scores': tf.float32,
 'detection_boxes': tf.float32,
 'num_detections': tf.float32,
 'detection_classes': tf.float32}

In [15]:
detection_model.output_shapes

{'detection_scores': TensorShape([None, 100]),
 'detection_boxes': TensorShape([None, 100, 4]),
 'num_detections': TensorShape([None]),
 'detection_classes': TensorShape([None, 100])}

In [35]:
def run_inference_for_single_image(model, image):
    image = np.asarray(image)
    input_tensor = tf.convert_to_tensor(image)
    input_tensor = input_tensor[tf.newaxis,...]
    output_dict = model(input_tensor)
    num_detections = int(output_dict.pop('num_detections'))
    output_dict = {key:value[0, :num_detections].numpy() 
                 for key,value in output_dict.items()}
    output_dict['num_detections'] = num_detections
    output_dict['detection_classes'] = output_dict['detection_classes'].astype(np.int64)
    if 'detection_masks' in output_dict:
        detection_masks_reframed = utils_ops.reframe_box_masks_to_image_masks(
               output_dict['detection_masks'], output_dict['detection_boxes'],
               image.shape[0], image.shape[1])      
        detection_masks_reframed = tf.cast(detection_masks_reframed > 0.5,tf.uint8)
        output_dict['detection_masks_reframed'] = detection_masks_reframed.numpy()
    return output_dict

In [38]:
def show_inference(model, image_path):
    image_np = image_path
    image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    output_dict = run_inference_for_single_image(model, image_np)
    vis_util.visualize_boxes_and_labels_on_image_array(
      image_np,
      output_dict['detection_boxes'],
      output_dict['detection_classes'],
      output_dict['detection_scores'],
      category_index,
      instance_masks=output_dict.get('detection_masks_reframed', None),
      use_normalized_coordinates=True,
      line_thickness=2, min_score_thresh=0.5 )
    
    classes=output_dict['detection_classes'].astype(np.int64)
    scores=output_dict['detection_scores']
    #name = [category_index.get(class_) for class_, score in zip(classes, scores) if score > 0.8]
    #engin = pyttsx3.init('sapi5')
    
    #for i in range(0,len(name)):
        
     #   engin.say(name[i]['name'])
      #  engin.runAndWait()
        
    #print(name)
    
    image_np=cv2.cvtColor(image_np,cv2.COLOR_BGR2RGB)
    
    return image_np
   

In [34]:
!pip install opencv-python



In [40]:
#for image....
import cv2
img = cv2.imread(r'18.jpg')
img=show_inference(detection_model,img)
cv2.imshow('IMG',img)
cv2.waitKey(0)
cv2.destroyAllWindows()
#for video.....
#import cv2

#video=cv2.VideoCapture(r'D:\videos\Songs\videoplayback.mp4')

#while(True):
 #   ret,img=video.read()
  #  img=show_inference(detection_model,img)
   # cv2.imshow('LIVE',img)
    #k=cv2.waitKey(1)
    
 #   if(k==27):
  #      break
#cv2.destroyAllWindows()